### Task Schedule:
1. 訓練模型,調整參數(預計使用lgb，速度較快)(更:使用catboost,效果較好)
2. 嘗試使用不同模型,做Ensamble(blending, stacking)
3. Anomaly detection

### 注意事項:
1. 因為test data和train data時間不相關,在驗證時採取前60天訓練61~90天驗證,但仍需小心時間差異造成的影響
2. Anomaly detection: 看這類的模型能不能取代boosting(似乎是不行，盜刷數據並沒有那麼Anomaly）,但可以嘗試將Anomaly結果當成新feature

### <font color=green>Results:</font>

#### Catboost:
    * FE1~4,catboost訓練 validation:0.5, LB:0.55
    * FE1,3,4 validation:0.5313149, LB:0.6(th=0.37), default parameter

#### LGB:
    * 不做處理,直接丟lgb訓練 leaderboard score:0.45

## 讀取,轉換字串成可以訓練的資料

In [21]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
import math

import lightgbm as lgb
from lightgbm.sklearn import LGBMClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from catboost import CatBoostClassifier, Pool

import xgboost as xgb
from lightgbm.sklearn import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model  import LogisticRegression
from sklearn import svm
from sklearn.ensemble  import  GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB 

%matplotlib inline
data_path = '../data'

random_seed = 20
import json
path = '../code/para_dict/data_list.json'
with open(path,'r',encoding='utf-8') as f:
    para = json.loads(f.read())
    
data_list= para['data_list_FE_AN3']
delete_list = para['delete_list_overfit1']

## 除掉一些可能會overfit,distribution不同,受時間影響大的feature

delete_list1 = ['bacno','locdt','loctm','cano','fraud_ind']
delete_list2 = ['mchno','acqic','mcc']
delete_list3 = ['stocn','scity','csmcu']
delete_list4 = ['iterm']
delete_list6 = ['mchno_fraud_mean','mcc_fraud_mean','acqic_fraud_mean']
delete_list7 = ['bacno_locdt_skew','bacno_locdt_kurt','cano_locdt_skew','cano_locdt_kurt']
delete_list8 = ['bacno_lastlocdt','cano_lastlocdt']

delete_list5 = ['contp','etymd','hcefg','insfg','ovrlt','flbmk','flg_3dsmk']
# bacno_cano_nunique
# txkey

In [22]:
def load_data(data_list):
    data=[]
    for d in data_list:
        x = pd.read_csv('../data/preprocess/{}'.format(d))
        x_null = x.isnull().sum()
        
        print('\n',d,x.shape)
        print("Null columns:\n",x_null[x_null>0])

        if (d=='FE_data1.csv') or (d=='FE_data2.csv'):
            x.fillna(value=-1,inplace=True)

        data.append(x)

    all_data = pd.concat(data,axis=1)
    all_data_numsum = all_data.isnull().sum()
    print('ALL data shape:',all_data.shape)
    print('ALL data null:')
    print(all_data_numsum[all_data_numsum>0])
    return all_data

all_data = load_data(data_list)


 raw_data.csv (1943452, 23)
Null columns:
 fraud_ind    421665
dtype: int64

 FE_data1.csv (1943452, 56)
Null columns:
 cano_conam_skew      92612
cano_conam_kurt     155720
cano_conam_var       38678
bacno_locdt_skew     58303
bacno_locdt_kurt    101191
cano_locdt_skew      92612
cano_locdt_kurt     155720
dtype: int64

 FE_data2.csv (1943452, 30)
Null columns:
 Series([], dtype: int64)

 FE_data2_2.csv (1943452, 31)
Null columns:
 Series([], dtype: int64)

 FE_data3.csv (1943452, 9)
Null columns:
 Series([], dtype: int64)

 FE_data4.csv (1943452, 4)
Null columns:
 Series([], dtype: int64)

 FE_data4_2.csv (1943452, 3)
Null columns:
 Series([], dtype: int64)

 FE_data5.csv (1943452, 4)
Null columns:
 Series([], dtype: int64)

 FE_data6.csv (1943452, 17)
Null columns:
 Series([], dtype: int64)

 FE_data7.csv (1943452, 33)
Null columns:
 Series([], dtype: int64)

 FE_data7_2.csv (1943452, 33)
Null columns:
 Series([], dtype: int64)

 pca_feature.csv (1943452, 5)
Null columns:
 Series([

In [23]:
for c in all_data.columns:
    if c[-4:]=='mode':
#         print(c)
        pass

    print(c)

acqic
bacno
cano
conam
contp
csmcu
ecfg
etymd
flbmk
flg_3dsmk
fraud_ind
hcefg
insfg
iterm
locdt
loctm
mcc
mchno
ovrlt
scity
stocn
stscd
txkey
bacno_cano_nunique
bacno_txkey_nunique
conam_log
bacno_mean_conam
bacno_scale_conam
cano_mean_conam
cano_scale_conam
cano_conam_skew
cano_conam_kurt
cano_conam_mean
cano_conam_var
bacno_max_conam
bacno_ismax_conam
bacno_min_conam
bacno_ismin_conam
bacno_ratio_ecfg
cano_ratio_ecfg
locdt_week
loctm_hr
bacno_locdt_skew
bacno_locdt_kurt
cano_locdt_skew
cano_locdt_kurt
bacno_stocn_mode
bacno_scity_mode
bacno_csmcu_mode
bacno_stocn_ismode
bacno_scity_ismode
bacno_csmcu_ismode
cano_stocn_mode
cano_scity_mode
cano_csmcu_mode
cano_stocn_ismode
cano_scity_ismode
cano_csmcu_ismode
mchno_fraud_mean
mcc_fraud_mean
acqic_fraud_mean
mchno_bacno_nunique
acqic_bacno_nunique
mcc_bacno_nunique
stocn_bacno_nunique
scity_bacno_nunique
csmcu_bacno_nunique
mchno_cano_nunique
acqic_cano_nunique
mcc_cano_nunique
stocn_cano_nunique
scity_cano_nunique
csmcu_cano_nunique
mc

In [ ]:
print("Memory size of '"+str1+"' = "+str(sys.getsizeof(all_data))+ " bytes")

In [24]:
category_list=['csmcu','hcefg','stscd','scity','stocn','mcc','acqic','mchno','etymd','contp','locdt_week']
for c in all_data.columns:
    if c[-4:]=='mode':
        category_list.append(c)
        
all_data[category_list]=all_data[category_list].astype('category')

In [25]:
## 切三種不同的訓練集驗證

X_train1 = all_data[all_data['locdt']<=60].drop(columns=delete_list)
y_train1 = all_data[all_data['locdt']<=60]['fraud_ind']
X_test1 = all_data[(all_data['locdt']>60) & (all_data['locdt']<=90)].drop(columns=delete_list)
y_test1 = all_data[(all_data['locdt']>60) & (all_data['locdt']<=90)]['fraud_ind']

X_train2 = all_data[all_data['locdt']<=45].drop(columns=delete_list)
y_train2 = all_data[all_data['locdt']<=45]['fraud_ind']
X_test2 = all_data[(all_data['locdt']>45) & (all_data['locdt']<=90)].drop(columns=delete_list)
y_test2 = all_data[(all_data['locdt']>45) & (all_data['locdt']<=90)]['fraud_ind']

X_train3 = all_data[all_data['locdt']<=30].drop(columns=delete_list)
y_train3 = all_data[all_data['locdt']<=30]['fraud_ind']
X_test3 = all_data[(all_data['locdt']>30) & (all_data['locdt']<=90)].drop(columns=delete_list)
y_test3 = all_data[(all_data['locdt']>30) & (all_data['locdt']<=90)]['fraud_ind']


test_data_txkey = all_data[all_data['locdt']>90]['txkey'].copy().values
X_train_all = all_data[all_data['locdt']<=90].drop(columns=delete_list) 
y_train_all = all_data[all_data['locdt']<=90]['fraud_ind'] 

X_test_all = all_data[all_data['locdt']>90].drop(columns=delete_list) 
# y_test_all = all_data[all_data['locdt']>90]['fraud_ind'] 

## Train on catboost
* https://catboost.ai/docs/concepts/python-reference_parameters-list.html
* 研究有哪些可以用的function

In [26]:
categorical_features_indices = np.where(X_train1.columns.isin(category_list))[0]
print(X_train1.dtypes[categorical_features_indices])


acqic                 category
contp                 category
csmcu                 category
etymd                 category
hcefg                 category
mcc                   category
mchno                 category
scity                 category
stocn                 category
stscd                 category
locdt_week            category
bacno_stocn_mode      category
bacno_scity_mode      category
bacno_csmcu_mode      category
bacno_stocn_ismode    category
bacno_scity_ismode    category
bacno_csmcu_ismode    category
cano_stocn_mode       category
cano_scity_mode       category
cano_csmcu_mode       category
cano_stocn_ismode     category
cano_scity_ismode     category
cano_csmcu_ismode     category
bacno_contp_mode      category
cano_contp_mode       category
bacno_etymd_mode      category
cano_etymd_mode       category
bacno_mchno_mode      category
cano_mchno_mode       category
bacno_acqic_mode      category
cano_acqic_mode       category
bacno_mcc_mode        category
cano_mcc

In [27]:
param_cat={
    'loss_function':'Logloss',
    'eval_metric':'F1',
    
    'iterations':3000,
    'learning_rate':0.1,
    'l2_leaf_reg':3,
    'bagging_temperature':1,
#     'sampling_frequency':'PerTreeLevel',
    
    'depth':6,
    'one_hot_max_size':300,
    
#     'min_data_in_leaf':1,
#     'max_leaves':31,
#     'task_type':"GPU",
#     'devices':1',
#     'cat_features':categorical_features_indices,
    'thread_count':2,
    'rsm':1,
    'scale_pos_weight':1,
    'target_border':0.5,
    'random_seed':random_seed,
    'verbose':True    
}

In [ ]:
model = CatBoostClassifier(**param_cat)

model.fit(
    X_train1, y_train1,
    cat_features=categorical_features_indices,    
    eval_set=(X_test1, y_test1),
    early_stopping_rounds=300,
#     use_best_model=True,
    silent=False,
#     plot=True,
)
print('Model is fitted: ' + str(model.is_fitted()))
print('Model params:')
print(model.get_params())


# preds_class = model.predict(test_data)
# preds_proba = model.predict_proba(test_data)

0:	learn: 0.6025125	test: 0.3275013	best: 0.3275013 (0)	total: 1.9s	remaining: 1h 35m 1s
1:	learn: 0.6047178	test: 0.3304384	best: 0.3304384 (1)	total: 3.67s	remaining: 1h 31m 33s
2:	learn: 0.5741342	test: 0.3219436	best: 0.3304384 (1)	total: 5.32s	remaining: 1h 28m 34s
3:	learn: 0.5806873	test: 0.3323849	best: 0.3323849 (3)	total: 7.05s	remaining: 1h 28m
4:	learn: 0.5915905	test: 0.3392059	best: 0.3392059 (4)	total: 8.68s	remaining: 1h 26m 38s
5:	learn: 0.6002957	test: 0.3494366	best: 0.3494366 (5)	total: 10.5s	remaining: 1h 27m 7s
6:	learn: 0.6123220	test: 0.3736236	best: 0.3736236 (6)	total: 12.3s	remaining: 1h 27m 33s
7:	learn: 0.6133270	test: 0.3821802	best: 0.3821802 (7)	total: 14.1s	remaining: 1h 27m 44s
8:	learn: 0.6183352	test: 0.3860887	best: 0.3860887 (8)	total: 15.7s	remaining: 1h 26m 43s
9:	learn: 0.6252309	test: 0.3917098	best: 0.3917098 (9)	total: 17.5s	remaining: 1h 27m 18s
10:	learn: 0.6175570	test: 0.3936128	best: 0.3936128 (10)	total: 19.3s	remaining: 1h 27m 17s
11:	

89:	learn: 0.7734941	test: 0.5168024	best: 0.5194970 (83)	total: 2m 37s	remaining: 1h 24m 58s
90:	learn: 0.7739067	test: 0.5176231	best: 0.5194970 (83)	total: 2m 39s	remaining: 1h 24m 58s
91:	learn: 0.7742932	test: 0.5186880	best: 0.5194970 (83)	total: 2m 41s	remaining: 1h 24m 54s
92:	learn: 0.7751249	test: 0.5193552	best: 0.5194970 (83)	total: 2m 42s	remaining: 1h 24m 51s
93:	learn: 0.7753543	test: 0.5194673	best: 0.5194970 (83)	total: 2m 44s	remaining: 1h 24m 45s
94:	learn: 0.7757545	test: 0.5233692	best: 0.5233692 (94)	total: 2m 46s	remaining: 1h 24m 43s
95:	learn: 0.7768680	test: 0.5244118	best: 0.5244118 (95)	total: 2m 48s	remaining: 1h 24m 51s
96:	learn: 0.7767375	test: 0.5245445	best: 0.5245445 (96)	total: 2m 49s	remaining: 1h 24m 47s
97:	learn: 0.7783558	test: 0.5271924	best: 0.5271924 (97)	total: 2m 51s	remaining: 1h 24m 48s
98:	learn: 0.7790361	test: 0.5271259	best: 0.5271924 (97)	total: 2m 53s	remaining: 1h 24m 40s
99:	learn: 0.7803252	test: 0.5260771	best: 0.5271924 (97)	to

In [ ]:
# 585
# 593
# 不同的thread不同的結果 乾==

In [ ]:
# bacno_loctm_diff
# bacno_locdt_diff
# cano_etymd_diff2

In [14]:
train_pool=Pool(X_test1, y_test1)#,cat_features=categorical_features_indices)
feature_importances = model.get_feature_importance(train_pool)
feature_names = X_test1.columns
for score, name in sorted(zip(feature_importances, feature_names), reverse=True):
    print('{}: {}'.format(name, score))

stocn_bacno_nunique: 4.27147772608887
stocn_cano_nunique: 3.9461634419327383
cano_locdt_skew: 2.396508069173645
isolationtree_all_feature2: 2.335635605106094
isolationtree_all_feature4: 1.9139562535494108
bacno_locdt_skew: 1.7838422590934975
bacno_mean_conam: 1.5943031823517535
cano_conam_skew: 1.4941550254626028
cano_lastlocdt2: 1.4581720008445553
mchno_bacno_nunique: 1.4330180527102077
ecfg: 1.4145051957624744
cano_locdt_kurt: 1.3730570888702105
kmeans_all_feature5: 1.328464174396649
cano_change_ratio: 1.2945591971551271
txkey: 1.285063675896271
acqic_ratio_cano: 1.2799474003008426
bacno_locdt_kurt: 1.26177654622445
kmeans_all_feature: 1.2207630944612773
acqic: 1.1918626117767168
scity: 1.1675668260260523
stocn_value_counts: 1.1553414687966435
bacno_1_count: 1.1351224341472128
isolationtree_all_feature: 1.0732567718808348
acqic_value_counts: 1.0116493921642393
bacno_cano_changetimes: 1.0083832481785664
conam: 0.9711697828390357
cano_conam_kurt: 0.9709167361939064
pca2_feature_1: 0.96

In [ ]:
## 理論上th設0.5一定是最好的？

y_test1_pred = model.predict_proba(X_test1,verbose=True)[:,1]
th=0.37

y_test1_pred[y_test1_pred>th]=1
y_test1_pred[y_test1_pred<=th]=0
print('F1 score',f1_score(y_test1, y_test1_pred))

tn, fp, fn, tp = confusion_matrix(y_test1, y_test1_pred).ravel()
print('tn fp fn tp')
print(tn, fp, fn, tp)
print('Percision', tp/(tp+fp))
print('Recall',tp/(tp+fn))

np.save('../data/preprocess/y_test1_pred.npy',y_test1_pred)

In [ ]:
model = CatBoostClassifier(**param_cat)

model.fit(
    X_train_all, y_train_all,
#     cat_features=categorical_features_indices,    
    silent=False
)


In [ ]:
y_test_pred_cat = model.predict_proba(X_test_all)[:,1]

print(X_test_all.index)

th=0.37
y_test_pred_cat[y_test_pred_cat>th]=1
y_test_pred_cat[y_test_pred_cat<=th]=0

## write csv

In [ ]:
import time
print(time.localtime( time.time() ))
a = time.localtime( time.time() )
print(str(a.tm_mon)+str(a.tm_mday))

In [ ]:
result = y_test_pred_cat
test_data_txkey = all_data[all_data['locdt']>90]['txkey'].values

print('{}: prediction positive ratio'.format(result.sum()/result.shape[0]))
print('{}: training positive ratio'.format(y_train_all.sum()/y_train_all.shape[0]))

import time
t_now = time.localtime( time.time() )
t = str(t_now.tm_mon)+str(t_now.tm_mday)+str(t_now.tm_hour)+str(t_now.tm_min)
print('Now:',t)

submit_file_name='submit_cat_AN2_over1_th{}_time{}.csv'.format(th,t)
import csv
with open('../prediction/{}'.format(submit_file_name),'w') as f:
    writer = csv.writer(f)
    writer.writerow(['txkey','fraud_ind'])
    for i in range(result.shape[0]):
        writer.writerow([test_data_txkey[i], result[i]])
        
# with open('../prediction/log.txt','w') as f:
#     print('{}'.format(submit_file_name),file=f)
#     print('delete_list:\n{}'.format(delete_list),file=f)

## 贝叶斯调参
* http://nohup.cc/article/258/
* https://github.com/fmfn/BayesianOptimization

Results:
{'params': {'bagging_temperature': 1.0,
  'learning_rate': 0.2,
  'reg_lambda': 17.267046492047776},
 'target': 0.5339179149920837}

In [ ]:
# from bayes_opt import BayesianOptimization
# def cat_train(bagging_temperature, reg_lambda, learning_rate):
#     params = {
#         'iterations':2000,
#         'depth':6,
#         'bagging_temperature':bagging_temperature,
#         'reg_lambda':reg_lambda,
#         'learning_rate':learning_rate,
#         'loss_function':'Logloss',
#         'eval_metric':'F1',
#         'random_seed':random_seed,
#         'verbose':30
#     }
 
#     model = CatBoostClassifier(**params)
#     # 评价数据集是验证集，评价指标是AUC
#     model.fit(X_train1, y_train1,\
#               eval_set=(X_test1, y_test1),\
#               cat_features=categorical_features_indices,\
#               early_stopping_rounds=200) 
     
#     print(params)
#     score_max = model.best_score_.get('validation').get('F1')
#     return score_max
 
# cat_opt = BayesianOptimization(cat_train, 
#                            {
#                               'bagging_temperature': (1, 50),  
#                               'reg_lambda': (1, 200),
#                               'learning_rate':(0.05, 0.2)
#                             })
 
# cat_opt.maximize(n_iter=15, init_points=random_seed)
# cat_opt.max

## Permutation importance

In [ ]:
# def permutation_scoring(model,X,y):
#     y_pred = model.predict(X)
#     return f1_score(y, y_pred)

# import eli5
# from eli5.sklearn import PermutationImportance

# print(y_test1.sum())
# print(model.score(X_test1, y_test1))
# perm = PermutationImportance(model, random_state=random_seed,scoring=permutation_scoring,n_iter=50).fit(X_test1, y_test1)
# feature_importance1 = pd.DataFrame({'feature':X_test1.columns.tolist(),'importance':perm.feature_importances_})
# delete_col1 = feature_importance1.iloc[:,0][(feature_importance1['importance'].values)<=0.0000]
# print(delete_col1)

# feature_importance1.sort_values(by=['importance'],inplace=True)
# top_col = feature_importance1
# for i in range(top_col.shape[0]):
#     print(top_col.iloc[i].values)

In [ ]:
# from eli5.catboost import explain_weights_catboost
# explain_weights_catboost(model, vec=None, top=70, importance_type='PredictionValuesChange', feature_names=None, pool=None)

## 將使用者分類來訓練模型:
1. 根據cano的個數分類
2. 根據txkey的個數分類
3. 根據stocn的眾數分類

murmur: 感覺這方法怪怪的,tree模型應該就能涵蓋進去

In [ ]:
## txkey
txkey_qcut_id = pd.qcut(all_data['bacno_txkey_nunique'],3,labels=[0,1,2])

# print(all_data['bacno_txkey_nunique'])
# print(txkey_qcut_id)
models=[]
for i in range(3):
    print(i)

## 不同model

In [ ]:
def th_transform(th,y_test1_pred,y_test1):
    y_test1_pred[y_test1_pred>th]=1
    y_test1_pred[y_test1_pred<=th]=0
    print('F1 score',f1_score(y_test1, y_test1_pred))
    
    return y_test1_pred

In [ ]:
rf = RandomForestClassifier(n_estimators=200,random_state=random_seed,verbose=0,class_weight="balanced", criterion='gini')

rf.fit(X_train1, y_train1) 
y_pred_rf = rf.predict_proba(X_test1)[:,1]
th_transform(0.37,y_pred_rf,y_test1)

In [ ]:
def lgb_f1_score(y_true, y_pred):
    y_pred = np.round(y_pred) # scikits f1 doesn't like probabilities
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    print()
    print('tn, fp, fn, tp')
    print(tn, fp, fn, tp)
    return 'f1', f1_score(y_true, y_pred), True

param_dist_lgb = {
                  'num_leaves':26, 
#                   'max_depth':-1, 
                  'learning_rate':0.1, 
                  'n_estimators':2000,
                  'objective': 'binary',
#                   'subsample': 1, 
#                   'colsample_bytree': 0.5, 
                  'lambda_l1': 10,
                  'lambda_l2': 10,
#                   'min_child_weight': 1,
                  'random_state': random_seed,
                 }
evals_result = {}

lgb_clf = LGBMClassifier(**param_dist_lgb)
lgb_clf.fit(X_train1, y_train1,
        eval_set=[(X_train1, y_train1),(X_test1, y_test1)],
        eval_metric=lgb_f1_score,
        early_stopping_rounds=50,
        verbose=True,
        callbacks=[lgb.record_evaluation(evals_result)]
        )
y_test_pred = lgb_clf.predict(X_test1)
print('F1 score',f1_score(y_test1, y_test_pred))

## Anomaly detection
* one class svm
* isolation tree
* replicator NN
* Kmeans?
* KNN(take too much time)

## 異常偵測
wiki<br>
https://zh.wikipedia.org/wiki/%E5%BC%82%E5%B8%B8%E6%A3%80%E6%B5%8B#cite_note-9

因為盜刷很可能都是outlier，一般的機器學習方法在outlier上表現會很差，因此可以用來解釋為什麼會train不好的原因
http://www.cainiaoxueyuan.com/suanfa/7017.html<br>
https://towardsdatascience.com/outlier-detection-with-isolation-forest-3d190448d45e<br>
https://medium.com/@cyeninesky3/oneclass-svm-%E7%95%B0%E5%B8%B8%E6%AA%A2%E6%B8%AC%E4%BB%BB%E5%8B%99-anomaly-detection-%E7%9A%84%E7%AE%97%E6%B3%95%E7%90%86%E8%A7%A3%E8%88%87%E5%AF%A6%E8%B8%90-cf5f0bbb01c0<br>

isolation tree<br>
https://zhuanlan.zhihu.com/p/25040651
https://scikit-learn.org/stable/auto_examples/ensemble/plot_isolation_forest.html
https://towardsdatascience.com/outlier-detection-with-extended-isolation-forest-1e248a3fe97b

oneclass svm<br>
https://scikit-learn.org/stable/auto_examples/svm/plot_oneclass.html

Replicator NN<br>
https://togaware.com/papers/dawak02.pdf

one class kmeans?<br>
https://ai100-2.cupoy.com/mission/D57

## 製作特徵
XGB, LGB, PCA, Isolation Forest, Kmean距離？, oneclass SVM?
當作新feature

In [ ]:
# import xgboost as xgb
# param_dist_xgb = {'learning_rate':0.01, #默认0.3
#               'n_estimators':1000, #树的个数
# #               'max_depth':5,
# #               'min_child_weight':1,
# #               'gamma':0.2,
# #               'subsample':0.8,
# #               'colsample_bytree':0.8,
# #               'objective': 'binary:logistic', #逻辑回归损失函数
# #               'nthread':4,  #cpu线程数
# #               'scale_pos_weight':1,
#               'seed':random_seed}  #随机种子

# evals_result = {}

# xgb_clf = xgb.XGBClassifier(**param_dist_xgb)
# xgb_clf.fit(X_train, y_train,
#         eval_set=[(X_train, y_train),(X_test, y_test)],
#         eval_metric=lgb_f1_score,
#         early_stopping_rounds=600,
#         verbose=True,
# #         callbacks=[xgb.record_evaluation(evals_result)]
#         )

# print('F1',f1_score(y_test, xgb_clf.predict(X_test)))
# xgb_X_train = xgb_clf.apply(X_train)
# xgb_X_test = xgb_clf.apply(X_test)

## Train on LGB(未調參數)(效果不好)

In [ ]:
def lgb_f1_score(y_true, y_pred):
    y_pred = np.round(y_pred) # scikits f1 doesn't like probabilities
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    print()
    print('tn, fp, fn, tp')
    print(tn, fp, fn, tp)
    return 'f1', f1_score(y_true, y_pred), True

param_dist_lgb = {
#                   'num_leaves':45, 
#                   'max_depth':5, 
                  'learning_rate':0.1, 
                  'n_estimators':2000,
                  'objective': 'binary',
#                   'subsample': 1, 
#                   'colsample_bytree': 0.5, 
#                   'lambda_l1': 0.1,
#                   'lambda_l2': 0,
#                   'min_child_weight': 1,
                  'random_state': random_seed,
                 }
evals_result = {}

lgb_clf = LGBMClassifier(**param_dist_lgb)
lgb_clf.fit(X_train1, y_train1,
        eval_set=[(X_train1, y_train1),(X_test1, y_test1)],
        eval_metric=lgb_f1_score,
        early_stopping_rounds=50,
        verbose=True,
        callbacks=[lgb.record_evaluation(evals_result)]
        )
y_test_pred = lgb_clf.predict(X_test1)

In [ ]:
print('Plotting metrics recorded during training...')
ax = lgb.plot_metric(evals_result, metric='f1')
plt.show()

print('Plotting feature importances...')
ax = lgb.plot_importance(lgb_clf, max_num_features=30)
plt.show()

print('Plotting 4th tree...')  # one tree use categorical feature to split
ax = lgb.plot_tree(lgb_clf, tree_index=3, figsize=(15, 15), show_info=['split_gain'])
plt.show()

print('Plotting 4th tree with graphviz...')
graph = lgb.create_tree_digraph(lgb_clf, tree_index=3, name='Tree4')
graph.render(view=True)

In [ ]:
feature_importance = np.stack([X_train1.columns.values,lgb_clf.feature_importances_]).transpose()
feature_importance = pd.DataFrame(feature_importance,columns=['feature_name','importance'])
feature_importance.sort_values(by=['importance'],inplace=True,ascending=False)
print(feature_importance)

## PCA visualization in one person who has fraud data

In [ ]:
from sklearn.decomposition import PCA
def PCA_plot(x,label):
    x = x.drop(columns=delete_list)
    
    ## 應該先轉dummy,標準化,再PCA
#     dummy_list=['contp','etymd','stscd','hcefg']
#     dummy_list2=['stocn','scity','csmcu']#'mchno','acqic','mcc',
#     x[dummy_list] = x[dummy_list].astype(object)
#     x[dummy_list2] = x[dummy_list2].astype(object)
#     x = pd.get_dummies(x)    
    
    from sklearn.preprocessing import StandardScaler 
    stdsc = StandardScaler() 
    x = stdsc.fit_transform(x)
    print(x.shape,label.sum())

    PCA_model = PCA(n_components=2)
    train_data_pca = PCA_model.fit_transform(x)
    train_data_pca1 = train_data_pca[label==1]
    train_data_pca0 = train_data_pca[label==0]
    
    plt.clf()
    plt.figure(figsize=(10,10))
    plt.scatter(train_data_pca1[:, 0], train_data_pca1[:, 1], c='r',label='fraud transaction',s=100)
    plt.scatter(train_data_pca0[:, 0], train_data_pca0[:, 1], c='b',label='normal transaction',s=3)
    plt.legend()
    plt.show()
    
bacno_hasfraud = all_data[all_data['fraud_ind']==1]['bacno'].unique()
print(bacno_hasfraud.shape[0])
print(all_data[all_data['fraud_ind']==1].shape[0])

for i in range(bacno_hasfraud.shape[0]):
    if all_data[all_data['bacno']==bacno_hasfraud[i]].shape[0]>300:
        print('Ploting PCA on bacno-{}'.format(bacno_hasfraud[i]))
        PCA_plot(all_data[all_data['bacno']==bacno_hasfraud[i]],all_data[all_data['bacno']==bacno_hasfraud[i]]['fraud_ind'])

## TSNE, Kmeans作圖?

## Isolation Forest

In [ ]:
from sklearn.ensemble import IsolationForest

c_ratio = y_train.sum()/y_train.shape[0]
# fit the model
clf = IsolationForest(behaviour='new', max_samples=0.8, max_features=1,
                      random_state=random_seed, contamination=c_ratio)
clf.fit(X_train)

y_pred_train = clf.predict(X_train)
y_pred_test = clf.predict(X_test)

y_pred_test2 = -y_pred_test
y_pred_test2[y_pred_test2==-1]=0
y_pred_test2.sum()

y_pred_train2 = -y_pred_train
y_pred_train2[y_pred_train2==-1]=0
y_pred_train2.sum()

from sklearn.metrics import f1_score
print(f1_score(y_train, y_pred_train2))
print(f1_score(y_test, y_pred_test2))

isolationtree_X_train = clf.score_samples(X_train)
isolationtree_X_test = clf.score_samples(X_test)

print(isolationtree_X_train)

## One class SVM

In [ ]:
from sklearn import svm

clf = svm.OneClassSVM(nu=0.1, kernel="rbf", gamma='scale',verbose=True, random_state=random_seed)
clf.fit(X_train)
y_pred_train = clf.predict(X_train)
y_pred_test = clf.predict(X_test)
y_pred_test.sum()

y_pred_train2 = -y_pred_train
y_pred_train2[y_pred_train2==-1]=0
y_pred_train2.sum()

from sklearn.metrics import f1_score
print(f1_score(y_train, y_pred_train2))
print(f1_score(y_test, y_pred_test2))

svm_X_train = clf.score_samples(X_train)
svm_X_test = clf.score_samples(X_test)

print(isolationtree_X_train)

In [ ]:
# cano_lastlocdt
# bacno_lastlocdt
# bacno_cano_nunique



In [ ]:
# bacno_etymd_nunique
# bacno_contp_nunique

In [ ]:
# 用hinge loss(當SVM)

In [ ]:
# X_train['cents']
# encoding data

# GroupKfold
# vanilla KFold